# This is the code for Quantum Optimization

In [91]:
import unittest
import qiskit
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
import numpy as np
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as SamplerV2
import operator
from qiskit.primitives import Sampler, Estimator
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz
import qiskit_aer as Aer
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from scipy.optimize import minimize
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.converters import InequalityToEquality
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter
import matplotlib
matplotlib.use('TkAgg')  # Set non-interactive backend
import matplotlib.pyplot as plt
from qiskit import transpile
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import os
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator as Estimator3
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.optimizers import COBYLA

In [92]:
pip show qiskit

Name: qiskit
Version: 1.4.0
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: /home/maop7/.conda/envs/QuantumQiskit/lib/python3.9/site-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-aer-gpu, qiskit-algorithms, qiskit-ibm-runtime, qiskit-optimization
Note: you may need to restart the kernel to use updated packages.


## We first create the Docplex/Cplex optimization problem

In [93]:
opt_model = Model(name="MIP Model")

Nc= 5 # Nc is the number of seconds

Dist=2 # Distance to travel

tolerance = 0 # Tolerance in distance travelled

delta_v=1 # Rate of acceleration/deceleration set to 1

vmax=2 # Max speed of a TGV in France (in m/s)

alpha=0.05 # Regenerative braking efficiency

"""
We define two binary variables for two bits. 
When x=0 and y=0 then constant velocity
When x=1 and y=0 then acceleration
When x=0 and y=1 then breaking
"""

x={}
for i in range(0, Nc):
    x[i]= opt_model.binary_var(name=f"x_{i}")

y={}
for i in range(0, Nc):
    y[i]= opt_model.binary_var(name=f"y_{i}")

'''z={}
for i in range(0, Nc):
    z[i]= opt_model.binary_var(name=f"z_{i}")'''
    

objective = opt_model.linear_expr()
## objective is the hamiltonian/energy value we want to minimize
## Energy:
for i in range(0, Nc):
    '''objective += (delta_v**2) * x[i] - alpha*(delta_v**2)*y[i]'''
    objective += (delta_v**2)*x[i]

    
opt_model.minimize(objective)

In [94]:
## Constraint 1: (simultaneous braking/acceleration)


'''for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= x[i] , f"z_u_d_{i}") 

for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= y[i], f"z_p_d_{i}")
        
for i in range(0, Nc):
    opt_model.add_constraint(z[i] >= x[i]+ y[i] -1 , f"z_u_p_d_{i}") 

for i in range(0,Nc):
    opt_model.add_constraint(z[i] == 0, "No simultaneous braking or acceleration constraint"+str(i))
opt_model.add_constraint(opt_model.sum(z[i] for i in range(0, Nc)) == 0 , "No_simultaneous_braking_or_acceleration_constraint")
'''

for i in range(0, Nc):
    opt_model.add_constraint(x[i]+ y[i] -1 <= 0 , f"simultaneous_braking_acceleration_constraint_{i}") 
#### Print the optimization model info

opt_model.print_information()
opt_model.prettyprint()

Model: MIP Model
 - number of variables: 10
   - binary=10, integer=0, continuous=0
 - number of constraints: 5
   - linear=5
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool x_3;
dvar bool x_4;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;
dvar bool y_3;
dvar bool y_4;

minimize
 x_0 + x_1 + x_2 + x_3 + x_4;
 
subject to {
 simultaneous_braking_acceleration_constraint_0:
  x_0 + y_0 -1 <= 0;
 simultaneous_braking_acceleration_constraint_1:
  x_1 + y_1 -1 <= 0;
 simultaneous_braking_acceleration_constraint_2:
  x_2 + y_2 -1 <= 0;
 simultaneous_braking_acceleration_constraint_3:
  x_3 + y_3 -1 <= 0;
 simultaneous_braking_acceleration_constraint_4:
  x_4 + y_4 -1 <= 0;

}


In [95]:
"""
Constraint 2: (Total Distance constraints)
"""

distance = opt_model.linear_expr()
velocity = 0
for i in range(0, Nc):
    velocity = velocity + delta_v*(x[i]-y[i])
    distance += velocity
'''opt_model.add_constraint(distance <= Dist+tolerance, "Max_Distance_constraint")
opt_model.add_constraint(distance >= Dist-tolerance, "Min_Distance_constraint")'''
opt_model.add_constraint(distance == Dist, "Distance_constraint")

#### Print the optimization model info

opt_model.print_information() 

Model: MIP Model
 - number of variables: 10
   - binary=10, integer=0, continuous=0
 - number of constraints: 6
   - linear=6
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [96]:
"""
Constraint 3: (Net-Zero contraint)

"""
opt_model.add_constraint(opt_model.sum((y[i]-x[i]) for i in range(0, Nc)) == 0 , "Net_Zero_constraint")

#### Print the optimization model info

opt_model.print_information()

Model: MIP Model
 - number of variables: 10
   - binary=10, integer=0, continuous=0
 - number of constraints: 7
   - linear=7
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [97]:
"""
Constraint 4: (Maximum Speed)

"""
opt_model.add_constraint(opt_model.sum((delta_v*(x[i]) for i in range(0, Nc))) <= vmax , "Maximum_Speed_constraint")

#### Print the optimization model info

opt_model.print_information() 

Model: MIP Model
 - number of variables: 10
   - binary=10, integer=0, continuous=0
 - number of constraints: 8
   - linear=8
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [98]:

#Constraint 5: (Positive Speed)
for i in range(Nc):
    opt_model.add_constraint(opt_model.sum((x[i]-y[i]) for i in range(0, i)) >= 0 , "Positive_Speed_constraint"+str(i))

    #### Print the optimization model

opt_model.print_information()


Model: MIP Model
 - number of variables: 10
   - binary=10, integer=0, continuous=0
 - number of constraints: 13
   - linear=13
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [99]:
'''
#Constraint 6: (Must leave immediately)

opt_model.add_constraint(x[0]==1, "Must leave immediately constraint")
    

#### Print the optimization model info

opt_model.print_information()
'''

'\n#Constraint 6: (Must leave immediately)\n\nopt_model.add_constraint(x[0]==1, "Must leave immediately constraint")\n    \n\n#### Print the optimization model info\n\nopt_model.print_information()\n'

In [100]:
#### Print the optimization model
print(opt_model.prettyprint())

// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool x_3;
dvar bool x_4;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;
dvar bool y_3;
dvar bool y_4;

minimize
 x_0 + x_1 + x_2 + x_3 + x_4;
 
subject to {
 simultaneous_braking_acceleration_constraint_0:
  x_0 + y_0 -1 <= 0;
 simultaneous_braking_acceleration_constraint_1:
  x_1 + y_1 -1 <= 0;
 simultaneous_braking_acceleration_constraint_2:
  x_2 + y_2 -1 <= 0;
 simultaneous_braking_acceleration_constraint_3:
  x_3 + y_3 -1 <= 0;
 simultaneous_braking_acceleration_constraint_4:
  x_4 + y_4 -1 <= 0;
 Distance_constraint:
  5 x_0 - 5 y_0 + 4 x_1 - 4 y_1 + 3 x_2 - 3 y_2 + 2 x_3 - 2 y_3 + x_4 - y_4 ==
  2;
 Net_Zero_constraint:
  y_0 - x_0 + y_1 - x_1 + y_2 - x_2 + y_3 - x_3 + y_4 - x_4 == 0;
 Maximum_Speed_constraint:
  x_0 + x_1 + x_2 + x_3 + x_4 <= 2;
 Positive_Speed_constraint0:
  0 >= 0;
 Positive_Speed_constraint1:
  x_0 - y_0 >= 0;
 Positiv

## Problem conversion

In [101]:
# Conversion to qad_model
qp_quad = from_docplex_mp(opt_model)
print("Number of variables:", len(qp_quad.variables))

Number of variables: 10


/tmp/ipykernel_34337/414607200.py:2: UserWarning: Trivial constraint: Positive_Speed_constraint0: 0 >= 0
  qp_quad = from_docplex_mp(opt_model)


In [102]:
# Conversion to qubo
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp_quad)

print(f"QUBO variables: {len(qubo.variables)}")
print(f"QUBO constraints: {len(qubo.linear_constraints)}")

'''opt_model.export_as_lp("qubo1.lp")'''

QUBO variables: 19
QUBO constraints: 0


'opt_model.export_as_lp("qubo1.lp")'

In [103]:
qubitOp, offset = qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))
num_qubits = qubitOp.num_qubits
print(num_qubits)
print(offset)

Offset: 1919.5
Ising Hamiltonian:
SparsePauliOp(['IIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIZIIIIIIII', 'IIIIIIIIIZIIIIIIIII', 'IIIIIIIIZIIIIIIIIII', 'IIIIIIIZIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIZIZ', 'IIIIIIIIIIIIIIIZIIZ', 'IIIIIIIIIIIIIIZIIIZ', 'IIIIIIIIIIIIIZIIIIZ', 'IIIIIIIIIIIIZIIIIIZ', 'IIIIIIIIIIIZIIIIIIZ', 'IIIIIIIIIIZIIIIIIIZ', 'IIIIIIIIIZIIIIIIIIZ', 'IIIIIIIIZIIIIIIIIIZ', 'IIIIIIIZIIIIIIIIIIZ', 'IIIIIIZIIIIIIIIIIIZ', 'IIIIIIZIIIIIIIIIIII', 'IIIIIZIIIIIIIIIIIIZ', 'IIIIIZIIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIIZ', 'IIIIZIIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIIIZ', 'IIIZIIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIIIZ', 'IIZIIIIIIIIIIIIIIII', 'IZIIIIIIIIIIIIIIIIZ', 'IZIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIIIZ', 'ZIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIIZIZI', 'IIIIIIIIIIIIIIZIIZI', 'IIIIIIIIIIIIIZIIIZI', 'IIIIIII

In [104]:
p=5

In [105]:
'''qubo_circuit.draw( style={'backgroundcolor': '#FFFFFF'})'''

"qubo_circuit.draw( style={'backgroundcolor': '#FFFFFF'})"

In [106]:
'''expanded_circuit = qubo_circuit.decompose()
expanded_circuit.draw( style={'backgroundcolor': '#FFFFFF'})'''

"expanded_circuit = qubo_circuit.decompose()\nexpanded_circuit.draw( style={'backgroundcolor': '#FFFFFF'})"

### Optimize circuit

In [107]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):

    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs
    total_cost = cost + offset

    objective_func_vals.append(total_cost)


    return total_cost

In [108]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<MY_IBM_QUANTUM_TOKEN>", overwrite=True, set_as_default=True)
init_params = []
for j in range(1,p):
    qubo_circuit = QAOAAnsatz(cost_operator=qubitOp, reps=j)
    qubo_circuit.measure_all()
    print(qubo_circuit.num_qubits)
    backend = AerSimulator(device='GPU')
    print(backend)

    # Create pass manager for transpilation
    pm = generate_preset_pass_manager(optimization_level=3,
                                        backend=backend)

    candidate_circuit = pm.run(qubo_circuit)
    '''candidate_circuit.draw()'''
    num_qubits_circ = candidate_circuit.num_qubits
    print(num_qubits_circ)

    import random
    initial_gamma = np.pi
    initial_beta = np.pi/2


    initial_gamma = np.pi * random.random()
    initial_beta = np.pi * random.random()
    init_params.append(initial_gamma)
    init_params.append(initial_beta)


    # Optimize circuit
    objective_func_vals = [] # Global variable
    with Session(backend=backend) as session:
        # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
        estimator = Estimator(mode=session)
        estimator.options.default_shots = 10000

        '''# Set simple error suppression/mitigation options, this is for when on quantum hardware
        estimator.options.dynamical_decoupling.enable = True
        estimator.options.dynamical_decoupling.sequence_type = "XY4"
        estimator.options.twirling.enable_gates = True
        estimator.options.twirling.num_randomizations = "auto"'''

        result = minimize(
            cost_func_estimator,
            init_params,
            args=(candidate_circuit, qubitOp, estimator),
            method="COBYLA",
            tol=1e-3,
        )
        print("Result : " + str(i) )
        print(result)
    init_params = [phase for phase in result.x]
    print(init_params)

    

19
AerSimulator('aer_simulator')


19
Result : 4
 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 1824.5941
       x: [ 2.774e+00  3.189e+00]
    nfev: 23
   maxcv: 0.0
[np.float64(2.7742427808678385), np.float64(3.1892152336031145)]
19
AerSimulator('aer_simulator')
19
Result : 4
 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 1669.83
       x: [ 3.952e+00  3.013e+00  6.971e-01  2.837e+00]
    nfev: 31
   maxcv: 0.0
[np.float64(3.9515633784503637), np.float64(3.013405025407164), np.float64(0.697058804172536), np.float64(2.8372504361653808)]
19
AerSimulator('aer_simulator')
19
Result : 4
 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 1875.2253
       x: [ 3.952e+00  4.013e+00  6.973e-01  3.837e+00  2.716e+00
            1.971e+00]
    nfev: 45
   maxcv: 0.0
[np.float64(3.951609814853316), np.float64(4.013178240199688), np.float64(0.6973339740893006), np.float64(3.837115118447191), np.float64(2.716302921870123), 

### Execute circuit

In [109]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

In [110]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)

In [111]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub= (optimized_circuit, )
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {key: val/shots for key, val in counts_int.items()}

final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
print(final_distribution_int)
print(final_distribution_bin)

{177529: 0.0001, 56920: 0.0001, 46086: 0.0001, 188552: 0.0001, 164697: 0.0001, 190356: 0.0001, 179520: 0.0001, 94210: 0.0001, 461323: 0.0001, 84137: 0.0001, 508366: 0.0001, 250438: 0.0001, 280680: 0.0001, 211502: 0.0001, 190729: 0.0001, 32601: 0.0001, 29319: 0.0001, 389342: 0.0001, 45191: 0.0001, 340368: 0.0001, 305374: 0.0001, 98882: 0.0001, 221332: 0.0001, 369473: 0.0001, 147944: 0.0001, 179993: 0.0001, 333400: 0.0001, 5023: 0.0001, 331262: 0.0001, 8252: 0.0001, 172297: 0.0001, 181523: 0.0001, 414397: 0.0001, 17143: 0.0001, 141608: 0.0001, 116985: 0.0001, 25970: 0.0001, 492410: 0.0001, 311845: 0.0001, 150535: 0.0001, 251300: 0.0001, 172208: 0.0001, 307342: 0.0001, 229730: 0.0002, 274024: 0.0001, 116953: 0.0001, 50479: 0.0001, 175748: 0.0002, 475051: 0.0001, 152270: 0.0001, 7624: 0.0001, 363111: 0.0001, 479396: 0.0001, 361902: 0.0001, 461489: 0.0001, 262192: 0.0001, 330679: 0.0001, 147487: 0.0001, 342680: 0.0001, 471935: 0.0001, 52460: 0.0001, 523861: 0.0001, 11969: 0.0001, 490881: 0.

/home/maop7/.conda/envs/QuantumQiskit/lib/python3.9/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:269: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [112]:
# Sort both distributions by probability in descending order
sorted_dist_int = dict(sorted(final_distribution_int.items(), key=lambda x: x[1], reverse=True))
sorted_dist_bin = dict(sorted(final_distribution_bin.items(), key=lambda x: x[1], reverse=True))

# Print top 10 most likely outcomes
top_bin_list = []
for k, v in list(sorted_dist_bin.items())[:50]:
    temp_list = list(k)
    top_bin_list.append([int(re) for re in temp_list])


In [113]:
def calculate_cost_from_qubo(bitstring):
    # Convert bitstring to a dictionary of variable assignments
    var_assignment = {var.name: bit for var, bit in zip(qubo.variables, bitstring)}
    
    # Evaluate the cost function
    cost = qubo.objective.evaluate(var_assignment)
    
    return cost

# Example usage
cost_l = []
for bin in top_bin_list:
    cost = calculate_cost_from_qubo(bin)
    cost_l.append(cost)

min_index, min_value = min(enumerate(cost_l), key=operator.itemgetter(1))
print(min_index)

most_likely_bitstring = top_bin_list[min_index]

24


### Post-process

In [114]:
x_value = most_likely_bitstring[0:Nc]
y_value = most_likely_bitstring[Nc:2*Nc]

print(x_value)
print(y_value)

[0, 1, 0, 0, 0]
[0, 0, 0, 0, 1]


### Visualization

In [115]:
def distance(x, y, Nc):
    velocity = 0
    vel = [0]
    dist = [0]
    dist_tot= 0
    for i in range(0, Nc):
        velocity = velocity + delta_v*(x[i]-y[i])
        if velocity < 0:
            velocity = -velocity
        vel.append(velocity)
        dist_tot += velocity
        dist.append(dist_tot)
    return dist,vel

In [116]:
time= np.arange(Nc+1)
distn, velo = distance(x_value, y_value, Nc)

In [117]:
plt.figure(figsize=(15,8))

matplotlib.rcParams.update({"font.size": 15})

plt.plot(time, velo, c='b', marker="o", markersize=2, linestyle='-')#, label='label)

plt.xlabel('Time')
plt.ylabel('Velocity')
plt.title("Velocity vs Time")
plt.grid(axis='x')
plt.grid(axis='y')
plt.legend()
plt.show()

/tmp/ipykernel_34337/2353725290.py:12: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [118]:
plt.figure(figsize=(10,6))

matplotlib.rcParams.update({"font.size": 15})

plt.plot(time, distn, c='b', marker="o", markersize=1, linestyle='-')#, label='label')
plt.xlabel('Time')
plt.ylabel('Distance')
plt.title("Distance vs Time")
plt.legend()
plt.show()

/tmp/ipykernel_34337/1348846773.py:9: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
invalid command name "131839256274368process_stream_events"
    while executing
"131839256274368process_stream_events"
    ("after" script)
